In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd

### Lectura datos - Recaudo Bogotá

Debido a la cantidad de registros (~56 millones), debo cargar y procesar el dataset en dos pasadas cada una de 23 millones de registros (la capacidad que soporta mi equipo). 

Es importante aclarar que de los 56 millones de registros, nos vamos a enfocar en sòlo aquellos que usaron servicio troncal y que pertenecen a algún tipo de población vulnerable (Adulto Mayor, Discapacidad, Apoyo Ciudadano). Esto debido a que 

* 1) los servicios zonales y duales no presentan una buena calidad en su recolección y 
* 2) esta población específico puede ser la más afectada por el virus

El archivo de los 56 millones de registros previamente fue filtrado para eliminar aquellos que no fueran servicios TRONCALES, esto con ayuda del comando **sed** de linux

In [29]:
df = pd.read_csv('TRONCAL2.txt',sep='\t')
                 #, nrows=23000000)

In [30]:
df.shape

(11208158, 15)

In [31]:
df.head()

,CLEARING_TEXTO,USO_TISC_TEXTO,Sistema,FASE,FCS__LINEA_ZONA,FCS_ESTACION_RUTABUS,ACCESOESTACION_PARADABUS,IDPOS,Serial_Alias,TISC_TIPO,TISC_PERFIL,TISC_VALOR_VALIDACION,TISC_SALDO_PREVIO,TISC_SALDO_POSTERIOR,Total_Validaciones
0,17/03/2020,17-03-2020 06:02:54,TRONCAL,Fase 3,(31) Zona F Av. Américas,(05100) Banderas P. Central,(LK) Peatonal (Occidente),10000766,202000548542228,(2) tullave Plus,(001) Adulto,2500,0,0,1
1,17/03/2020,17-03-2020 04:30:21,TRONCAL,Fase 3,(31) Zona F Av. Américas,(05100) Banderas P. Central,"(KA) ALIMENTACION K.CENTRAL, HOSPITAL, CORABA...",10000776,202000355737545,(1) tullave Básica,(001) Adulto,2500,0,0,1
2,17/03/2020,17-03-2020 04:36:54,TRONCAL,Fase 3,(31) Zona F Av. Américas,(05100) Banderas P. Central,"(KA) ALIMENTACION K.CENTRAL, HOSPITAL, CORABA...",10000776,202000632588260,(1) tullave Básica,(001) Adulto,2500,0,0,1
3,17/03/2020,17-03-2020 04:40:14,TRONCAL,Fase 3,(31) Zona F Av. Américas,(05100) Banderas P. Central,"(KA) ALIMENTACION K.CENTRAL, HOSPITAL, CORABA...",10000775,202000899061145,(1) tullave Básica,(001) Adulto,2500,0,0,1
4,17/03/2020,17-03-2020 04:57:08,TRONCAL,Fase 3,(31) Zona F Av. Américas,(05100) Banderas P. Central,"(KA) ALIMENTACION K.CENTRAL, HOSPITAL, CORABA...",10000775,202000645029193,(1) tullave Básica,(001) Adulto,2500,0,0,1


In [32]:
df.dtypes

CLEARING_TEXTO              object
USO_TISC_TEXTO              object
Sistema                     object
FASE                        object
FCS__LINEA_ZONA             object
FCS_ESTACION_RUTABUS        object
ACCESOESTACION_PARADABUS    object
IDPOS                        int64
Serial_Alias                 int64
TISC_TIPO                   object
TISC_PERFIL                 object
TISC_VALOR_VALIDACION        int64
TISC_SALDO_PREVIO            int64
TISC_SALDO_POSTERIOR         int64
Total_Validaciones           int64
dtype: object

In [33]:
df['Sistema'].value_counts()

TRONCAL    11208158
Name: Sistema, dtype: int64

In [34]:
df['TISC_TIPO'].value_counts()

(2) tullave Plus      6676810
(1) tullave Básica    4531348
Name: TISC_TIPO, dtype: int64

In [35]:
df['TISC_PERFIL'].value_counts()

(001) Adulto             9384889
(006) Apoyo Ciudadano     946184
(002) Adulto Mayor        546106
(005) Discapacidad        183156
(101) Adulto PV           147823
Name: TISC_PERFIL, dtype: int64

In [36]:
df['CLEARING_TEXTO'].value_counts()

13/03/2020    2211872
12/03/2020    1979689
07/03/2020    1684538
10/03/2020    1323633
02/03/2020    1048588
11/03/2020     774756
04/03/2020     723879
18/03/2020     653763
03/03/2020     323224
06/03/2020     270605
08/03/2020     139880
01/03/2020      64073
09/03/2020       3569
17/03/2020       3208
05/03/2020       1503
16/03/2020       1378
Name: CLEARING_TEXTO, dtype: int64

### Get población objetivo (adultos mayores, discapacidad y apoyo ciudadano)
Aqui filtraremos los servicios troncales para extraer los registros de la población objetivo

In [37]:
objetivo = df.loc[df['TISC_PERFIL'].isin(["(006) Apoyo Ciudadano","(002) Adulto Mayor","(005) Discapacidad"])]

In [38]:
objetivo['TISC_PERFIL'].value_counts()

(006) Apoyo Ciudadano    946184
(002) Adulto Mayor       546106
(005) Discapacidad       183156
Name: TISC_PERFIL, dtype: int64

In [39]:
objetivo.to_csv('dos.csv', index=False)

### Fusión de las dos mitades y enriquecimiento del dataset (add lat/long)

In [3]:
mitad1 = pd.read_csv('uno.csv')
mitad2 = pd.read_csv('dos.csv')
todo   = pd.concat([mitad1, mitad2])

In [4]:
todo.shape

(5225584, 15)

In [5]:
todo.head()

,CLEARING_TEXTO,USO_TISC_TEXTO,Sistema,FASE,FCS__LINEA_ZONA,FCS_ESTACION_RUTABUS,ACCESOESTACION_PARADABUS,IDPOS,Serial_Alias,TISC_TIPO,TISC_PERFIL,TISC_VALOR_VALIDACION,TISC_SALDO_PREVIO,TISC_SALDO_POSTERIOR,Total_Validaciones
0,17/03/2020,17-03-2020 06:16:52,TRONCAL,Fase 3,(40) Zona T Ciudad Bolívar,(40000) Cable Portal Tunal,(01) Acceso Tunel - Cable,10001329,202000749889537,(2) tullave Plus,(006) Apoyo Ciudadano,0,0,0,1
1,17/03/2020,17-03-2020 06:17:58,TRONCAL,Fase 3,(40) Zona T Ciudad Bolívar,(40000) Cable Portal Tunal,(01) Acceso Tunel - Cable,10001329,202000388001537,(2) tullave Plus,(006) Apoyo Ciudadano,0,2000,2000,1
2,17/03/2020,17-03-2020 06:18:21,TRONCAL,Fase 3,(40) Zona T Ciudad Bolívar,(40000) Cable Portal Tunal,(01) Acceso Tunel - Cable,10001329,202000932447599,(2) tullave Plus,(006) Apoyo Ciudadano,0,0,0,1
3,17/03/2020,17-03-2020 06:21:29,TRONCAL,Fase 3,(40) Zona T Ciudad Bolívar,(40000) Cable Portal Tunal,(01) Acceso Tunel - Cable,10001329,202000582878492,(2) tullave Plus,(006) Apoyo Ciudadano,0,2375,2375,1
4,17/03/2020,17-03-2020 06:24:35,TRONCAL,Fase 3,(40) Zona T Ciudad Bolívar,(40000) Cable Portal Tunal,(01) Acceso Tunel - Cable,10001329,202000244282985,(2) tullave Plus,(006) Apoyo Ciudadano,0,34450,34450,1


In [6]:
todo['TISC_PERFIL'].value_counts()

(006) Apoyo Ciudadano    2923126
(002) Adulto Mayor       1721155
(005) Discapacidad        581303
Name: TISC_PERFIL, dtype: int64

In [8]:
todo['CLEARING_TEXTO'].value_counts().sort_index()

01/03/2020     98925
02/03/2020    365878
03/03/2020    377595
04/03/2020    376338
05/03/2020    371944
06/03/2020    375702
07/03/2020    253436
08/03/2020    100803
09/03/2020    370869
10/03/2020    374920
11/03/2020    372075
12/03/2020    358548
13/03/2020    354230
14/03/2020    231426
15/03/2020     82162
16/03/2020    282933
17/03/2020    250581
18/03/2020    227219
Name: CLEARING_TEXTO, dtype: int64

In [11]:
todo['FCS_ESTACION_RUTABUS'].unique()
#df['FCS_ESTACION_RUTABUS'].value_counts()

array(['(40000) Cable Portal Tunal', '(09001) Molinos',
       '(07000) Portal Sur', '(09100) Calle 40 Sur', '(04100) Carrera 77',
       '(04004) La Granja', '(14005) Las Aguas',
       '(09110) Avenida Jimenez', '(14004) Museo del Oro',
       '(06109) Centro Memoria', '(06107) Ciudad Universitaria',
       '(09120) Calle 63', '(09115) Calle 34', '(09118) Marly',
       '(09123) Calle 76', '(09114) Calle 26', '(09119) Calle 57',
       '(09121) Las Flores', '(09116) Avenida 39', '(09122) Calle 72',
       '(09111) Calle 19', '(09117) Calle 45', '(09113) Calle 22',
       '(12001) Carrera 43', '(12004) Carrera 22', '(12003) Ricaurte',
       '(14001) La Sabana', '(12000) Puente Aranda',
       '(12007) Zona Industrial', '(12002) CDS - Carrera 32',
       '(06108) Concejo de Bogotá', '(06100) Av. Rojas',
       '(40002) Manitas', '(40001) Juan Pablo II',
       '(40003) Mirador del Paraiso', '(06105) Quinta Paredes',
       '(06001) Modelia', '(06002) Normandía', '(06102) Salitre El Gr

In [7]:
todo = todo[~todo['FCS_ESTACION_RUTABUS'].str.contains("Corral")]
todo = todo[~todo['FCS_ESTACION_RUTABUS'].str.contains("Virtual")]

en total resulta un dataset de ~5 millones de registros

In [8]:
todo.shape

(5196553, 15)

A continuación crearemos un diccionario donde el nombre de cada estación de TM se mapea a su lat/lon, esto con el fin de agregar ambos atributos al dataset y poder visualizar la información a través de mapas

In [9]:
lat_lon = {'(40000) Cable Portal Tunal': [4.57035491,-74.13981319],
'(09001) Molinos': [4.55683171,-74.12173166],
'(07000) Portal Sur': [4.59581727,-74.17003806],
'(09100) Calle 40 Sur': [4.57596786,-74.11952549],
'(04100) Carrera 77': [4.6984667,-74.09483848],
'(04004) La Granja': [4.6984667,-74.09483848],
'(14005) Las Aguas': [4.60266984,-74.06817769],
'(09110) Avenida Jimenez': [4.6031429,-74.07913104],
'(14004) Museo del Oro': [4.60118385,-74.07295785],
'(06109) Centro Memoria': [4.62042703,-74.07649587],
'(06107) Ciudad Universitaria': [4.63020415,-74.08297304],
'(09120) Calle 63': [4.64874321,-74.06458735],
'(09115) Calle 34': [4.62221301,-74.06958764],
'(09118) Marly': [4.63951515,-74.06637045],
'(09123) Calle 76': [4.66406733,-74.06107216],
'(09114) Calle 26': [4.61617136,-74.07241326],
'(09119) Calle 57': [4.64379318,-74.0655874],
'(09121) Las Flores': [4.65471326,-74.06308827],
'(09116) Avenida 39': [4.62859306,-74.06823856],
'(09122) Calle 72': [4.65931729,-74.06190821],
'(09111) Calle 19': [4.60809693,-74.07663494],
'(09117) Calle 45': [4.6333741,-74.06738051],
'(09113) Calle 22': [4.61212995,-74.07441784],
'(12001) Carrera 43': [4.62280517,-74.10141668],
'(12004) Carrera 22': [4.60981799,-74.08671226],
'(12003) Ricaurte': [4.61283803,-74.09009135],
'(14001) La Sabana': [4.60602494,-74.08251114],
'(12000) Puente Aranda': [4.62556721,-74.10461377],
'(12007) Zona Industrial': [4.62024214,-74.09846359],
'(12002) CDS - Carrera 32': [4.6173591,-74.0951455],
'(06108) Concejo de Bogotá': [4.62508709,-74.07960195],
'(06100) Av. Rojas': [4.66245401,-74.10910097],
'(40002) Manitas': [4.550287, -74.150832],
'(40001) Juan Pablo II': [4.555876, -74.147660],
'(40003) Mirador del Paraiso': [4.550142, -74.158808],
'(06105) Quinta Paredes': [4.63641847,-74.09232196],
'(06001) Modelia': [4.67432269,-74.11675293],
'(06002) Normandía': [4.6673226,-74.11231782],
'(06102) Salitre El Greco': [4.65130342,-74.10185856],
'(06106) Recinto Ferial': [4.63410021,-74.08920322],
'(06104) Gobernación': [4.64263932,-74.09621841],
'(06101) El Tiempo': [4.65590347,-74.10508264],
'(06103) CAN': [4.64671936,-74.09893248],
'(04003) Avenida Ciudad de Cali': [4.70249985,-74.10025325],
'(04104) Avenida 68': [4.68580261,-74.08035668],
'(04105) Carrera 53': [4.68193956,-74.07736461],
'(04101) Minuto de Dios': [4.69669576,-74.091492],
'(04002) Carrera 90': [4.70473089,-74.10458038],
'(04102) Avenida Boyaca': [4.69390171,-74.08698686],
'(04108) El Polo': [4.67224842,-74.06583827],
'(04001) Quirigua': [4.70653892,-74.10852651],
'(04106) Carrera 47': [4.67831351,-74.0737265],
'(04103) Las Ferias': [4.69014066,-74.08402878],
'(04000) Cabecera Calle 80': [4.70952296,-74.11052456],
'(10007) Las Nieves': [4.6066789,-74.07390385],
'(10002) Av. Primero de Mayo': [4.57572274,-74.09456268],
'(10009) Museo Nacional': [4.61410007,-74.0696221],
'(10008) San Diego': [4.61113974,-74.07136878],
'(10000) Portal 20 de Julio': [4.56535867,-74.09819184],
'(10005) Bicentenario': [4.59504984,-74.08110114],
'(03000) Portal Suba': [4.74653621,-74.09625691],
'(03005) SUBA - AV. BOYACA': [4.71988287,-74.07452333],
'(03001) LA CAMPIÑA': [4.74271815,-74.09119976],
'(03009) Av. Suba- Calle 116': [4.69918067,-74.06975627],
'(03002) SUBA - TV- 91': [4.73820209,-74.08603261],
'(03006) NIZA - CALLE 127': [4.71168779,-74.07222629],
'(03014) SAN MARTIN': [4.67670446,-74.06698829],
'(03012) SUBA - CALLE 95': [4.68428551,-74.06303712],
'(03010) PUENTE LARGO': [4.69319361,-74.06707121],
'(03013) RIO NEGRO': [4.67978148,-74.06508621],
'(03003) 21 ANGELES': [4.73513203,-74.08032644],
'(03004) GRATAMIRA': [4.72773094,-74.07459429],
'(03011) SUBA - CALLE 100': [4.69003157,-74.06540117],
'(06000) Portal Eldorado': [4.68113815,-74.12098887],
'(07111) NQS - RICAURTE': [4.61367605,-74.09211642],
'(07109) CAD': [4.6238871,-74.08372009],
'(07201) Guatoque -Veraguas': [4.60413498,-74.09485855],
'(07110) PALOQUEMAO': [4.61799507,-74.08858928],
'(07104) Simon Bolivar': [4.65706335,-74.07775374],
'(07108) Av. El Dorado': [4.63026213,-74.07976893],
'(07106) EL CAMPIN': [4.64345924,-74.07879784],
'(07103) AV. CHILE': [4.66689641,-74.07398957],
'(07105) MOVISTAR ARENA': [4.65033029,-74.07828379],
'(07107) U. NACIONAL': [4.63616818,-74.07937689],
'(07102) NQS - CALLE 75': [4.67043543,-74.07122446],
'(07200) Tygua-San José': [4.59977591,-74.08872537],
'(07101) CASTELLANA': [4.67692744,-74.06210113],
'(06111) Universidades': [4.60533686,-74.06686963],
'(10003) Ciudad Jardín': [4.58121677,-74.09042752],
'(10006) San Victorino': [4.60177387,-74.07685397],
'(10001) Country Sur': [4.57139572,-74.09850883],
'(05001) Patio Bonito': [4.6348279,-74.16271887],
'(05002) Tintal': [4.63996012,-74.15698977],
'(05100) Banderas P. Central': [4.63139347,-74.14723714],
'(05105) Pradera': [4.62859531,-74.11905023],
'(05103) Marsella': [4.62962336,-74.12858953],
'(05102) Av. Américas - Av. Boyacá': [4.6302734, -74.13458973],
'(05107) Américas - Cr.53': [4.62776126,-74.11111797],
'(05101) Mandalay - Cr.73': [4.63071043,-74.1400219],
'(05005) Transversal 86': [4.63480527, -74.15326284],
'(05000) Portal Américas': [4.6282829,-74.17128277],
'(07005) ALQUERIA': [4.59437209,-74.13441089],
'(07010) Bosa': [4.59693732,-74.17918435],
'(07001) PERDOMO': [4.59568824,-74.16341884],
'(07113) SANTA ISABEL': [4.600994,-74.10336685],
'(07112) COMUNEROS': [4.60570801,-74.09855967],
'(07006) General Santander': [4.59360505,-74.1282667],
'(07002) MADELENA': [4.59624121,-74.15725663],
'(07003) SEVILLANA': [4.59529816,-74.14767232],
'(07008) NQS - Calle 30 Sur': [4.59529801,-74.11581228],
'(07007) NQS - Calle 38A Sur': [4.59398803,-74.12294152],
'(57503) Ampliacion San Mateo': [4.58543136,-74.20659531],
'(07503) SAN MATEO': [4.58543136,-74.20659531],
'(07009) SENA': [4.597543,-74.11020509],
'(07004) VENECIA': [4.59571313,-74.14159012],
'(07506) DESPENSA': [4.59495435,-74.18746863],
'(02200) Alcalá': [4.72076577,-74.05144057],
'(02000) Cabecera Autopista Norte': [4.75447203,-74.04596522],
'(02105) Calle 142': [4.72662081,-74.05050551],
'(02304) Heroes': [4.66689435,-74.06050012],
'(02303) Calle 85': [4.67241739,-74.05941206],
'(02201) Prado': [4.71393371,-74.05258664],
'(02101) Toberín': [4.74733097,-74.04703429],
'(02102) Calle 161': [4.74154193,-74.04796835],
'(02103) Mazurén': [4.73547088,-74.04897041],
'(02300) Calle 100': [4.68434548,-74.05750794],
'(02104) Calle 146': [4.73064684,-74.04987147],
'(02202) Calle 127': [4.70524465,-74.05401073],
'(02001) Centro Comercial Santa Fe': [4.763451, -74.044344],
'(02205) Calle 106': [4.69280155,-74.05614686],
'(02502) Terminal': [4.76963114,-74.04340206],
'(02204) Pepe Sierra': [4.69785859,-74.05531281],
'(02302) Virrey': [4.67739343,-74.05858701],
'(08001) Parque el Tunal': [4.56841787,-74.13489203],
'(09002) Consuelo': [4.56016875,-74.12383771],
'(08002) Biblioteca': [4.57032486,-74.13025087],
'(09109) Tercer Milenio': [4.59799788,-74.08383022],
'(09105) Río Fucha': [4.58322682,-74.09893378],
'(09101) Quiroga': [4.57676184,-74.11424032],
'(09103) Olaya': [4.57893083,-74.10729408],
'(09003) Socorro': [4.56458879,-74.12553274],
'(09107) Hortúa': [4.59079985,-74.09021346],
'(09104) Restrepo': [4.58176282,-74.10152688],
'(09108) Hospital': [4.59486786,-74.08629931],
'(09004) Santa Lucía': [4.57071884,-74.12473269],
'(09106) Nariño': [4.58629883,-74.0938266],
'(08000) Portal Tunal': [4.57035491,-74.13981319],
'(09000) Cabecera Usme': [4.53079248,-74.12017173],
'(10010) San Bernardo': [4.58944781,-74.08474329],
'(10004) Policarpa': [4.58653979,-74.08655036],
'(04107) Escuela Militar': [4.67527646,-74.06923537],
'(03007) HUMEDAL CORDOBA': [4.70618274,-74.07099928],
'(07505) LEON XIII': [4.59201635,-74.19337284],
'(07504) TERREROS': [4.59003235,-74.19752499],
'(08100) Portal Tunal Cable': [4.570918, -74.140342]}

In [10]:
todo["lat"] = todo["FCS_ESTACION_RUTABUS"].apply(lambda x:lat_lon[x][0])
todo["lon"] = todo["FCS_ESTACION_RUTABUS"].apply(lambda x:lat_lon[x][1])

In [11]:
todo.shape

(5196553, 17)

In [12]:
todo.head()

,CLEARING_TEXTO,USO_TISC_TEXTO,Sistema,FASE,FCS__LINEA_ZONA,FCS_ESTACION_RUTABUS,ACCESOESTACION_PARADABUS,IDPOS,Serial_Alias,TISC_TIPO,TISC_PERFIL,TISC_VALOR_VALIDACION,TISC_SALDO_PREVIO,TISC_SALDO_POSTERIOR,Total_Validaciones,lat,lon
0,17/03/2020,17-03-2020 06:16:52,TRONCAL,Fase 3,(40) Zona T Ciudad Bolívar,(40000) Cable Portal Tunal,(01) Acceso Tunel - Cable,10001329,202000749889537,(2) tullave Plus,(006) Apoyo Ciudadano,0,0,0,1,4.570355,-74.139813
1,17/03/2020,17-03-2020 06:17:58,TRONCAL,Fase 3,(40) Zona T Ciudad Bolívar,(40000) Cable Portal Tunal,(01) Acceso Tunel - Cable,10001329,202000388001537,(2) tullave Plus,(006) Apoyo Ciudadano,0,2000,2000,1,4.570355,-74.139813
2,17/03/2020,17-03-2020 06:18:21,TRONCAL,Fase 3,(40) Zona T Ciudad Bolívar,(40000) Cable Portal Tunal,(01) Acceso Tunel - Cable,10001329,202000932447599,(2) tullave Plus,(006) Apoyo Ciudadano,0,0,0,1,4.570355,-74.139813
3,17/03/2020,17-03-2020 06:21:29,TRONCAL,Fase 3,(40) Zona T Ciudad Bolívar,(40000) Cable Portal Tunal,(01) Acceso Tunel - Cable,10001329,202000582878492,(2) tullave Plus,(006) Apoyo Ciudadano,0,2375,2375,1,4.570355,-74.139813
4,17/03/2020,17-03-2020 06:24:35,TRONCAL,Fase 3,(40) Zona T Ciudad Bolívar,(40000) Cable Portal Tunal,(01) Acceso Tunel - Cable,10001329,202000244282985,(2) tullave Plus,(006) Apoyo Ciudadano,0,34450,34450,1,4.570355,-74.139813


In [53]:
todo.to_csv('tolis.csv', index=False)

### AHORA SÍ A AGRUPAR Y VISUALIZAR LOS DATOS

#### agrupación de los datos según día / estación de origen del viaje / perfil

In [57]:
a = todo.groupby(['CLEARING_TEXTO','FCS_ESTACION_RUTABUS','TISC_PERFIL'])
data_frame = pd.DataFrame(a.size(),columns=['Count'])
print(data_frame)

                                                                        Count
CLEARING_TEXTO FCS_ESTACION_RUTABUS              TISC_PERFIL                 
01/03/2020     (02000) Cabecera Autopista Norte  (002) Adulto Mayor      2158
                                                 (005) Discapacidad       547
                                                 (006) Apoyo Ciudadano   2450
               (02001) Centro Comercial Santa Fe (002) Adulto Mayor       178
                                                 (005) Discapacidad        27
...                                                                       ...
18/03/2020     (40003) Mirador del Paraiso       (005) Discapacidad       109
                                                 (006) Apoyo Ciudadano   1266
               (57503) Ampliacion San Mateo      (002) Adulto Mayor       340
                                                 (005) Discapacidad        89
                                                 (006) Apoyo Ciu

#### con la siguiente linea puedo filtrar la información por días

In [53]:
dia = todo.loc[todo['CLEARING_TEXTO'] == "18/03/2020"]
dia

,CLEARING_TEXTO,USO_TISC_TEXTO,Sistema,FASE,FCS__LINEA_ZONA,FCS_ESTACION_RUTABUS,ACCESOESTACION_PARADABUS,IDPOS,Serial_Alias,TISC_TIPO,TISC_PERFIL,TISC_VALOR_VALIDACION,TISC_SALDO_PREVIO,TISC_SALDO_POSTERIOR,Total_Validaciones,lat,lon
2252809,18/03/2020,18-03-2020 03:59:23,TRONCAL,Fase 3,(34) Zona H Caracas Sur,(09001) Molinos,(01) Acceso Sur,10001192,202000356750941,(2) tullave Plus,(006) Apoyo Ciudadano,0,5050,5050,1,4.556832,-74.121732
2252810,18/03/2020,18-03-2020 04:09:06,TRONCAL,Fase 3,(34) Zona H Caracas Sur,(09001) Molinos,(01) Acceso Sur,10001192,202000441378629,(2) tullave Plus,(006) Apoyo Ciudadano,0,2950,2950,1,4.556832,-74.121732
2252811,18/03/2020,18-03-2020 04:10:03,TRONCAL,Fase 3,(34) Zona H Caracas Sur,(09001) Molinos,(01) Acceso Sur,10001192,202000833140417,(2) tullave Plus,(006) Apoyo Ciudadano,0,5175,5175,1,4.556832,-74.121732
2252812,18/03/2020,18-03-2020 04:12:07,TRONCAL,Fase 3,(34) Zona H Caracas Sur,(09001) Molinos,(01) Acceso Sur,10001192,202000544967214,(2) tullave Plus,(006) Apoyo Ciudadano,0,12100,12100,1,4.556832,-74.121732
2252813,18/03/2020,18-03-2020 04:12:45,TRONCAL,Fase 3,(34) Zona H Caracas Sur,(09001) Molinos,(01) Acceso Sur,10001192,202000186872716,(2) tullave Plus,(006) Apoyo Ciudadano,0,3725,3725,1,4.556832,-74.121732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1675441,18/03/2020,18-03-2020 18:04:11,TRONCAL,Fase 3,(33) Zona B AutoNorte,(02300) Calle 100,(01) Acceso Norte,10001078,202000451351469,(2) tullave Plus,(006) Apoyo Ciudadano,200,3225,3025,1,4.684345,-74.057508
1675442,18/03/2020,18-03-2020 18:14:27,TRONCAL,Fase 3,(33) Zona B AutoNorte,(02300) Calle 100,(01) Acceso Norte,10001078,202000782786735,(2) tullave Plus,(006) Apoyo Ciudadano,200,4750,4550,1,4.684345,-74.057508
1675443,18/03/2020,18-03-2020 19:08:01,TRONCAL,Fase 3,(33) Zona B AutoNorte,(02300) Calle 100,(01) Acceso Norte,10001078,202000181274787,(2) tullave Plus,(006) Apoyo Ciudadano,200,2700,2500,1,4.684345,-74.057508
1675444,18/03/2020,18-03-2020 19:38:07,TRONCAL,Fase 3,(33) Zona B AutoNorte,(02300) Calle 100,(01) Acceso Norte,10001078,202000422748723,(2) tullave Plus,(006) Apoyo Ciudadano,200,31800,31600,1,4.684345,-74.057508


In [54]:
dia.to_csv('dieciocho.csv')

### Nota: otras funciones útiles durante el proceso

In [ ]:
chunksize = 10
tfr = pd.read_csv('operador.txt', sep='\t', chunksize=chunksize, iterator=True)
df = pd.concat(tfr, ignore_index=True)

In [7]:
df["CLEARING_TEXTO"]=df['CLEARING_TEXTO'].apply(lambda x:x.replace("-","/"))